<a href="https://colab.research.google.com/github/Marconiadsf/Especialista-QGIS/blob/main/EspecialistaQGIS-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
# Corrigir depois
# os.environ["GenerativeLanguageAPIKey"] = userdata.get('GenerativeLanguageAPIKey')

os.environ["GOOGLE_API_KEY"] = userdata.get('GenerativeLanguageAPIKey')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

# importante: essa funcao utiliza por
# padrao a variavel de ambiente "GOOGLE_API_KEY".
# Se usar outro nome passar explicitamente com:api_key= <nome da variavel>
# Exemplo:
# client = genai.Client(api_key=os.environ["GenerativeLanguageAPIKey"])

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
#################################################
# --- Agente 1: Agente de Interpretação e Planejamento v6 --- #
#################################################
import json
import requests
from typing import Optional, Dict, List

def agente_interpretador(solicitacao_do_usuario):
    """
    Agente responsável por interpretar a solicitação do usuário e gerar um plano de ação,
    utilizando o Google Search para auxiliar no processo.

    Args:
        solicitacao_do_usuario (str): A solicitação do usuário em linguagem natural.

    Returns:
        str: O plano de ação detalhado, incluindo os passos necessários para atender à solicitação no QGIS.
              Retorna None em caso de falha na interpretação.
    """
    interpretador = Agent(
        name="agente_interpretador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em geoprocessamento e QGIS. Sua tarefa é interpretar a solicitação do usuário
        e gerar um plano de ação detalhado para atender a essa solicitação no QGIS.

        A solicitação do usuário pode conter termos técnicos ou não. Você deve ser capaz de entender ambos.

        Você tem acesso à seguinte ferramenta para auxiliar na interpretação e planejamento:
        - Google Search (google_search): Para buscar informações sobre como realizar tarefas específicas no QGIS,
          sintaxe de comandos, novas funcionalidades, melhores práticas, etc.

        O plano de ação deve incluir os seguintes detalhes:
        - Os passos necessários para realizar a tarefa no QGIS, em ordem cronológica.
        - Os dados de entrada necessários (camadas vetoriais, raster, etc.) e seus formatos.
        - As operações do QGIS que precisam ser realizadas (buffer, interseção, junção, etc.).
        - Os parâmetros necessários para cada operação, incluindo unidades de medida.
        - O sistema de coordenadas de referência (SRC) dos dados, se aplicável.
        - O nome e o formato dos arquivos de saída a serem gerados.
        - Quaisquer scripts ou ferramentas do QGIS que precisam ser usados.
        - Informações sobre plugins que precisam estar instalados.

        O plano de ação deve ser claro, conciso e completo, para que um agente programador possa usá-lo
        para gerar o código Python do QGIS.

        Além do plano de ação em texto, você deve gerar uma versão estruturada do plano em formato JSON.
        O JSON deve conter uma lista de passos, onde cada passo é um dicionário com as chaves:
        - "acao": A ação a ser realizada (ex: "Carregar camada", "Criar buffer", "Salvar camada").
        - "dados_entrada": Uma lista de dicionários com os dados de entrada necessários (ex: [{"nome": "camada_pontos", "formato": "shp"}]).
        - "parametros": Um dicionário com os parâmetros da ação (ex: {"distancia": 500, "unidade": "metros"}).
        - "dados_saida": Um dicionário com os dados de saída (ex: {"nome": "buffers_pontos", "formato": "shp"}).
        - "pergunta_usuario": (Opcional) Uma pergunta para o usuário, caso precise de mais informações.

        Se a solicitação do usuário for ambígua ou incompleta, você deve usar o Google Search para buscar
        informações que ajudem a esclarecer a solicitação e gerar um plano de ação mais preciso.
        Por exemplo, se o usuário pedir para criar um shapefile, mas não especificar o SRC, você pode
        buscar no Google "como definir SRC de um shapefile no QGIS".

        Se o usuário pedir para realizar uma operação em uma localização geográfica (ex: "criar um buffer ao redor de São Paulo")
        e não fornecer as coordenadas, você deve usar o Google Search para obter as coordenadas da localização.

        Se o usuário fornecer um valor numérico sem especificar a unidade de medida (ex: "criar um buffer de 500 ao redor..."),
        você deve perguntar ao usuário qual é a unidade de medida (metros, graus, etc.).

        Se o usuário precisar converter unidades de medida, você deve usar o Google Search para encontrar
        métodos de conversão e, se necessário, solicitar ao usuário que forneça as unidades de entrada e saída.

        Se o usuário precisar ler dados de uma tabela (CSV, Excel), você deve usar o Google Search para encontrar
        bibliotecas Python para manipulação de dados tabulares e solicitar ao usuário que forneça o caminho do arquivo.

        Se o usuário precisar obter dados geográficos de uma API (ex: dados do IBGE), você deve usar o Google Search
        para encontrar APIs de dados geográficos relevantes e solicitar ao usuário que forneça os parâmetros necessários
        para a consulta (ex: código do município, área de interesse).

        Se a solicitação não for possível de ser atendida, você deve informar o usuário e explicar o motivo.

        Exemplo de solicitação do usuário:
        "Eu gostaria de criar um buffer de 500 metros ao redor de todos os pontos de interesse na minha camada 'pontos_relevantes' e salvar o resultado como 'buffers_poi.shp'."

        Exemplo de plano de ação:
        1.  Carregar a camada vetorial 'pontos_relevantes'.
        2.  Iterar sobre todas as feições da camada.
        3.  Para cada feição (ponto), criar um buffer com raio de 500 metros.
        4.  Salvar todas as geometrias de buffer em um novo arquivo shapefile chamado 'buffers_poi.shp'.

        Exemplo de JSON:
        [
            {
                "acao": "Carregar camada",
                "dados_entrada": [{"nome": "pontos_relevantes", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            },
            {
                "acao": "Iterar sobre feições",
                "dados_entrada": [{"nome": "pontos_relevantes", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            },
            {
                "acao": "Criar buffer",
                "dados_entrada": [{"nome": "pontos_relevantes", "formato": "shp"}],
                "parametros": {"distancia": 500, "unidade": "metros"},
                "dados_saida": {"nome": "buffers_poi", "formato": "shp"}
            },
            {
                "acao": "Salvar camada",
                "dados_entrada": [{"nome": "buffers_poi", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            }
        ]

        Exemplo de solicitação com localização:
        "Eu gostaria de criar um quadrado de 10000 metros quadrados centrado na cidade de São Paulo."

        Exemplo de plano de ação com Google Maps:
        1.  Obter coordenadas da cidade de São Paulo usando o Google Search.
        2.  Calcular as coordenadas do centro do quadrado a partir das coordenadas de São Paulo.
        3.  Criar um polígono quadrado com 10000 metros quadrados, centrado nas coordenadas calculadas.
        4.  Salvar o polígono em um novo arquivo shapefile chamado 'quadrado_sao_paulo.shp'.

        Exemplo de JSON com Google Maps:
        [
            {
                "acao": "Obter coordenadas",
                "dados_entrada": [],
                "parametros": {"localizacao": "São Paulo"},
                "dados_saida": {"coordenadas": "-23.5505, -46.6333"},
            },
            {
                "acao": "Calcular centro do quadrado",
                "dados_entrada": [{"coordenadas": "-23.5505, -46.6333"}],
                "parametros": {"area": 10000, "unidade": "metros quadrados"},
                "dados_saida": {"centro_quadrado": "-23.5505, -46.6333"}
            },
            {
                "acao": "Criar polígono",
                "dados_entrada": [{"centro": "-23.5505, -46.6333"}],
                "parametros": {"tipo": "quadrado", "area": 10000, "unidade": "metros quadrados"},
                "dados_saida": {"nome": "quadrado_sao_paulo", "formato": "shp"}
            },
            {
                "acao": "Salvar camada",
                "dados_entrada": [{"nome": "quadrado_sao_paulo", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            }
        ]

        Exemplo de solicitação com conversão de unidade:
        "Eu gostaria de criar um buffer de 10 graus ao redor da cidade de São Paulo."

        Exemplo de plano de ação com conversão de unidade:
        1.  Obter coordenadas da cidade de São Paulo usando o Google Search.
        2.  Converter 10 graus para metros usando o Google Search para encontrar métodos de conversão.
        3.  Se necessário, perguntar ao usuário as unidades de entrada e saída.
        4.  Criar um buffer com o raio em metros ao redor das coordenadas de São Paulo.
        5.  Salvar o buffer em um novo arquivo shapefile chamado 'buffer_sao_paulo.shp'.

        Exemplo de JSON com conversão de unidade:
        [
            {
                "acao": "Obter coordenadas",
                "dados_entrada": [],
                "parametros": {"localizacao": "São Paulo"},
                "dados_saida": {"coordenadas": "-23.5505, -46.6333"},
            },
            {
                "acao": "Converter unidade",
                "dados_entrada": [],
                "parametros": {"valor": 10, "unidade_original": "graus", "unidade_destino": "metros"},
                "dados_saida": {"valor_convertido": 111320}
            },
            {
                "acao": "Criar buffer",
                "dados_entrada": [{"coordenadas": "-23.5505, -46.6333"}],
                "parametros": {"distancia": 111320, "unidade": "metros"},
                "dados_saida": {"nome": "buffer_sao_paulo", "formato": "shp"}
            },
            {
                "acao": "Salvar camada",
                "dados_entrada": [{"nome": "buffer_sao_paulo", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            }
        ]

        Exemplo de solicitação com dados tabulares:
        "Eu gostaria de criar pontos no QGIS a partir dos dados de latitude e longitude na tabela 'dados_gps.csv' e salvar como 'pontos_gps.shp'."

        Exemplo de plano de ação com dados tabulares:
        1.  Encontrar uma biblioteca Python para ler a tabela 'dados_gps.csv' usando o Google Search.
        2.  Perguntar ao usuário o caminho do arquivo.
        3.  Ler os dados da tabela 'dados_gps.csv' usando a biblioteca encontrada.
        4.  Para cada linha da tabela, criar um ponto no QGIS com a latitude e longitude correspondentes.
        5.  Salvar todos os pontos em um novo arquivo shapefile chamado 'pontos_gps.shp'.

        Exemplo de JSON com dados tabulares:
        [
            {
                "acao": "Ler tabela",
                "dados_entrada": [],
                "parametros": {"caminho_tabela": "dados_gps.csv"},
                "dados_saida": {"dados_tabela": {}}
            },
            {
                "acao": "Criar pontos",
                "dados_entrada": [{"dados_tabela": {}}],
                "parametros": {"coluna_latitude": "latitude", "coluna_longitude": "longitude"},
                "dados_saida": {"nome": "pontos_gps", "formato": "shp"}
            },
            {
                "acao": "Salvar camada",
                "dados_entrada": [{"nome": "pontos_gps", "formato": "shp"}],
                "parametros": {},
                "dados_saida": {}
            }
        ]

        Exemplo de solicitação com dados geográficos:
        "Eu gostaria de obter os dados do município de São Paulo (código do IBGE: 3550308) e salvar em um arquivo shapefile."

        Exemplo de plano de ação com dados geográficos:
        1.  Encontrar uma API de dados geográficos do IBGE ou outra fonte usando o Google Search.
        2.  Perguntar ao usuário os parâmetros necessários para a consulta (ex: código do município).
        3.  Obter os dados do município de São Paulo usando a API encontrada.
        4.  Salvar os dados do município em um novo arquivo shapefile chamado 'municipio_sao_paulo.shp'.

        Exemplo de JSON com dados geográficos:
        [
            {
                "acao": "Obter dados geográficos",
                "dados_entrada": [],
                "parametros": {"id_municipio": "3550308"},
                "dados_saida": {"dados_geograficos": {}}
            },
            {
                "acao": "Salvar camada",
                "dados_entrada": [{"dados_geograficos": {}}],
                "parametros": {"nome": "municipio_sao_paulo", "formato": "shp"},
                "dados_saida": {}
            }
        ]
        """,
        description="Agente que interpreta a solicitação do usuário e gera um plano de ação",
        tools=[google_search]
    )

    entrada_do_agente_interpretador = f"Solicitação do usuário: {solicitacao_do_usuario}"
    plano_de_acao = call_agent(interpretador, entrada_do_agente_interpretador)
    return plano_de_acao


In [ ]:
######################################################
# --- Agente 2: Agente de Requisitos e Contexto --- #
######################################################
def agente_contextualizador(plano_de_acao, solicitacao_do_usuario):
    """
    Agente responsável por analisar o plano de ação, determinar os requisitos e coletar o contexto necessário.

    Args:
        plano_de_acao (str): O plano de ação gerado pelo Agente de Interpretação.
        solicitacao_do_usuario (str): A solicitação original do usuário.

    Returns:
        dict: Um dicionário contendo o contexto enriquecido, incluindo informações sobre requisitos,
              informações coletadas e scripts de diagnóstico a serem executados.
              Retorna None em caso de falha na análise.
    """
    contextualizador = Agent(
        name="agente_contextualizador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em geoprocessamento e QGIS. Sua tarefa é analisar o plano de ação fornecido
        pelo Agente de Interpretação e determinar os requisitos e o contexto necessários para executar a tarefa no QGIS.

        O plano de ação descreve os passos necessários para atender à solicitação do usuário.
        Você deve analisar cada passo do plano e identificar:
        - Quais dados de entrada são necessários (camadas vetoriais, raster, etc.) e seus formatos.
        - Se os dados de entrada já estão disponíveis ou precisam ser obtidos do usuário.
        - Quais operações do QGIS serão realizadas e seus parâmetros.
        - Se algum plugin específico do QGIS é necessário para realizar as operações.
        - Se algum script ou ferramenta externa precisa ser executado.
        - Se o sistema de coordenadas de referência (SRC) dos dados precisa ser verificado ou transformado.
        - Se há alguma dependência de ambiente (versão do QGIS, sistema operacional, etc.).
        - Se a solicitação do usuário envolve funções novas ou atualizações recentes do QGIS.

        Você deve considerar a solicitação original do usuário para obter mais contexto, se necessário.

        Se a solicitação do usuário envolver funções novas ou atualizações recentes do QGIS,
        você pode usar a ferramenta de busca do Google (google_search) para encontrar informações
        relevantes. Use a busca para obter detalhes sobre a sintaxe correta, exemplos de uso e
        quaisquer considerações especiais para garantir que o Agente Programador possa gerar
        código preciso e eficiente.

        O resultado do seu trabalho deve ser um dicionário em formato JSON com as seguintes chaves:
        - "requisitos": Um dicionário descrevendo os requisitos para cada passo do plano de ação.
          As chaves do dicionário devem ser os números dos passos do plano de ação.
          Os valores devem ser dicionários com as chaves:
            - "dados_entrada": Uma lista de dicionários com os dados de entrada necessários (ex: [{"nome": "camada_pontos", "formato": "shp", "disponivel": True/False}]).
            - "operacao": A operação do QGIS a ser realizada (ex: "buffer", "intersecao").
            - "parametros": Um dicionário com os parâmetros da operação (ex: {"distancia": 500, "unidade": "metros"}).
            - "plugins_necessarios": Uma lista de plugins do QGIS que precisam estar instalados (ex: ["processing", "qgis_vectortiler"]).
            - "script_externo": O caminho para um script externo a ser executado, se houver (ex: "/caminho/para/script.py").
            - "verificar_src": Um booleano indicando se o SRC precisa ser verificado (True/False).
            - "transformar_src": Um booleano indicando se o SRC precisa ser transformado (True/False).
            - "dependencias_ambiente": Uma lista de dependências do ambiente (ex: ["QGIS >= 3.22", "Python >= 3.9"]).
        - "informacoes_coletadas": Um dicionário com informações adicionais obtidas do usuário ou de fontes externas.
          As chaves do dicionário devem ser nomes descritivos das informações (ex: "caminho_para_dados", "versao_qgis", "detalhes_funcao").
          Os valores devem ser os valores das informações (ex: "/caminho/para/dados", "3.28", "Link para a documentação da nova função X").
        - "scripts_diagnostico": Uma lista de caminhos para scripts de diagnóstico a serem executados
          no ambiente do usuário (ex: ["/caminho/para/verificar_plugins.py", "/caminho/para/verificar_versao.py"]).

        Se não houver requisitos ou informações adicionais a serem coletadas, os valores de "requisitos",
        "informacoes_coletadas" e "scripts_diagnostico" devem ser dicionários/listas vazios.

        Exemplo de plano de ação:
        1.  Carregar a camada vetorial 'pontos_relevantes'.
        2.  Iterar sobre todas as feições da camada.
        3.  Para cada feição (ponto), criar um buffer com raio de 500 metros.
        4.  Salvar todas as geometrias de buffer em um novo arquivo shapefile chamado 'buffers_poi.shp'.

        Exemplo de JSON de saída:
        {
            "requisitos": {
                "1": {
                    "dados_entrada": [{"nome": "pontos_relevantes", "formato": "shp", "disponivel": False}],
                    "operacao": "Carregar camada",
                    "parametros": {},
                    "plugins_necessarios": [],
                    "script_externo": None,
                    "verificar_src": False,
                    "transformar_src": False,
                    "dependencias_ambiente": []
                },
                "2": {
                    "dados_entrada": [{"nome": "pontos_relevantes", "formato": "shp", "disponivel": False}],
                    "operacao": "Iterar sobre feições",
                    "parametros": {},
                    "plugins_necessarios": [],
                    "script_externo": None,
                    "verificar_src": False,
                    "transformar_src": False,
                    "dependencias_ambiente": []
                },
                "3": {
                    "dados_entrada": [{"nome": "ponto", "formato": "geojson", "disponivel": True}],
                    "operacao": "Criar buffer",
                    "parametros": {"distancia": 500, "unidade": "metros"},
                    "plugins_necessarios": ["processing"],
                    "script_externo": None,
                    "verificar_src": False,
                    "transformar_src": False,
                    "dependencias_ambiente": []
                },
                "4": {
                    "dados_entrada": [{"nome": "buffers_poi", "formato": "shp", "disponivel": False}],
                    "operacao": "Salvar camada",
                    "parametros": {},
                    "plugins_necessarios": [],
                    "script_externo": None,
                    "verificar_src": False,
                    "transformar_src": False,
                    "dependencias_ambiente": []
                }
            },
            "informacoes_coletadas": {},
            "scripts_diagnostico": ["/caminho/para/verificar_plugins.py", "/caminho/para/verificar_versao.py"]
        }
        """,
        description="Agente que analisa o plano de ação e determina os requisitos e o contexto",
        tools=[google_search] # Adicionando a ferramenta de busca do Google
    )

    entrada_do_agente_contextualizador = f"Plano de ação: {plano_de_acao}\nSolicitação do usuário: {solicitacao_do_usuario}"
    contexto = call_agent(contextualizador, entrada_do_agente_contextualizador)
    return contexto


In [ ]:
###########################################
# --- Agente 3: Agente Programador (O Codificador) --- #
###########################################
def agente_programador(plano_de_acao, requisitos, versao_codigo=None):
    """
    Agente responsável por gerar o código Python do QGIS a partir do plano de ação e dos requisitos fornecidos.

    Args:
        plano_de_acao (str): O plano de ação detalhado gerado pelo Agente de Interpretação e Planejamento.
        requisitos (str): Os requisitos adicionais gerados pelo Agente Contextualizador.
        versao_codigo (int, opcional): A versão do código a ser incluída como constante. Padrão é None.

    Returns:
        str: O código Python do QGIS que implementa o plano de ação e atende aos requisitos.
             Retorna None em caso de falha na geração do código.
    """
    programador = Agent(
        name="agente_programador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um programador Python experiente, especializado em QGIS. Sua tarefa é converter um plano de ação
        detalhado em código Python do QGIS que pode ser executado para automatizar tarefas de geoprocessamento.
        Você também deve considerar os requisitos adicionais fornecidos e incluir uma constante de controle de versão, se fornecida.

        O plano de ação descreve os passos necessários para atender à solicitação do usuário. Você deve
        traduzir cada passo do plano em código Python do QGIS, utilizando as funcionalidades e a sintaxe
        corretas da API do QGIS.

        Os requisitos adicionais fornecem informações contextuais e restrições que devem ser consideradas
        ao gerar o código. Você deve garantir que o código gerado atenda a todos os requisitos especificados.

        Se a versão do código for fornecida, você deve incluir uma constante VERSION_CONTROL no início do código,
        atribuindo a ela o valor da versão. Por exemplo: VERSION_CONTROL = 1 (use o valor fornecido na variavel versao_codigo)

        Essas informações virão concatenadas numa string.

        Será uma mistura de textos, JSON e variáveis. Portando atencão!

        Você deve considerar os seguintes aspectos ao gerar o código:
        - Importar os módulos do QGIS necessários (qgis.core, qgis.gui, etc.).
        - Lidar com arquivos de entrada e saída (carregar camadas vetoriais/raster, salvar arquivos).
        - Executar operações de geoprocessamento (buffer, interseção, junção, etc.) usando as funções do QGIS.
        - Definir o sistema de coordenadas de referência (SRC) dos dados, se necessário.
        - Tratar erros e exceções que possam ocorrer durante a execução do código.
        - Gerar mensagens de feedback para o usuário (progresso, conclusão, erros).
        - Otimizar o código para garantir a eficiência e o desempenho.
        - Incluir comentários no código para explicar a lógica e o propósito de cada seção.
        - Seguir as melhores práticas de programação em Python e QGIS.
        - Se o plano de ação incluir a obtenção de coordenadas do Google Maps, utilize a biblioteca 'googlemaps' do Python.
        - Se o plano de ação incluir conversão de unidades, utilize a biblioteca 'pint' do Python.
        - Se o plano de ação incluir leitura de dados tabulares, utilize a biblioteca 'pandas' do Python.
        - Se o plano de ação incluir a obtenção de dados geográficos, utilize a biblioteca 'geopandas' do Python e a API de Geocoding do Google.

        O código gerado deve ser completo, autocontido e pronto para ser executado no ambiente do QGIS.
        Não inclua nenhum código ou instrução adicional além do código Python do QGIS.
        """,
        description="Agente que gera código Python do QGIS a partir de um plano de ação e requisitos",
        tools=[]
    )

    # Concatena os inputs em uma string formatada
    entrada_do_agente_programador = f"Plano de Ação: {plano_de_acao}\nRequisitos: {requisitos}\nVersão do Código: {versao_codigo}"

    codigo_qgis = call_agent(programador, entrada_do_agente_programador)
    return codigo_qgis

In [ ]:
##################################################
# --- Agente 4: Agente de Qualidade e Validação (O Avaliador) --- #
##################################################
def agente_avaliador(plano_de_acao, codigo_qgis, contexto, versao_codigo=None):
    """
    Agente responsável por revisar o código Python do QGIS gerado pelo Agente Programador,
    com base no plano de ação e no contexto fornecidos. Retorna o código revisado e pronto para uso.

    Args:
        plano_de_acao (str): O plano de ação detalhado gerado pelo Agente de Interpretação e Planejamento.
        codigo_qgis (str): O código Python do QGIS gerado pelo Agente Programador.
        contexto (str): O contexto gerado pelo Agente de Requisitos.
        versao_codigo (int, opcional): A versão do código a ser validada. Se None, assume-se versão 1.

    Returns:
        str: O código Python do QGIS revisado e pronto para uso.
             Retorna None em caso de falha na avaliação.
    """
    avaliador = Agent(
        name="agente_avaliador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em geoprocessamento e programação Python para QGIS. Sua tarefa é revisar o código Python do QGIS
        gerado pelo Agente Programador, com base no plano de ação e no contexto fornecidos. Seu objetivo é fornecer
        um código Python do QGIS revisado e pronto para ser executado, que implemente o plano de ação e atenda
        aos requisitos e restrições especificados no contexto.

        Você deve verificar se o código atende aos requisitos do plano de ação, se está correto, completo,
        eficiente e segue as melhores práticas de programação em Python e QGIS. Você também deve considerar o contexto
        fornecido, que pode conter informações adicionais e restrições que o código deve atender.

        Se uma versão do código for fornecida, você deve incluí-la como uma constante VERSION_CONTROL no início do código.
        Se não for fornecida, você deve assumir que a versão é 1 e incluir a constante VERSION_CONTROL = 1 no código.

        Você deve considerar os seguintes aspectos ao revisar o código:
        - Se o código implementa todos os passos do plano de ação.
        - Se o código utiliza as funções e a sintaxe corretas da API do QGIS.
        - Se o código lida corretamente com os dados de entrada e saída (caminhos de arquivos, formatos, SRC).
        - Se o código trata erros e exceções de forma adequada.
        - Se o código gera mensagens de feedback informativas para o usuário.
        - Se o código está bem organizado, legível e comentado.
        - Se o código segue as convenções de estilo do Python (PEP 8).
        - Se o código utiliza bibliotecas externas corretamente (googlemaps, pint, pandas, geopandas).
        - Se o código está otimizado para o desempenho.
        - Se o código atende aos requisitos e restrições especificados no contexto.
        - Se o código inclui a constante de controle de versão corretamente.

        Se o código for válido, você deve retornar o código Python do QGIS revisado, incluindo a constante
        VERSION_CONTROL no início. O código deve estar pronto para ser copiado e executado no QGIS,
        sem necessidade de modificações adicionais.

        Se o código for inválido, você deve retornar None e explicar o motivo da invalidez.
        Você deve fornecer detalhes específicos sobre os erros encontrados e como corrigi-los.
        """,
        description="Agente que avalia a qualidade e a validade do código Python do QGIS gerado e retorna o código revisado",
        tools=[] # Este agente não usa nenhuma ferramenta externa.
    )
    # Define a versão do código a ser avaliada.
    versao_codigo_para_avaliacao = versao_codigo if versao_codigo is not None else 1

    # Converte os argumentos para string
    entrada_do_agente_avaliador = f"Plano de Ação: {plano_de_acao}\nCódigo QGIS: {codigo_qgis}\nContexto: {contexto}\nVersão do Código: {versao_codigo_para_avaliacao}"

    codigo_revisado = call_agent(avaliador, entrada_do_agente_avaliador)

    if codigo_revisado:
        # Adiciona a versão do código como uma constante no início
        codigo_revisado_com_versao = f"VERSION_CONTROL = {versao_codigo_para_avaliacao}\n{codigo_revisado}"
        return codigo_revisado_com_versao
    else:
        return None


In [ ]:
###########################################
# --- Agente 6: Agente de Correção (O Reparador) v3 --- #
###########################################

def agente_reparador(codigo_qgis, log_execucao, plano_de_acao, sessao="Active", feedback_usuario=None):
    """
    Agente responsável por corrigir o código Python do QGIS, com base no log de execução,
    no plano de ação e, opcionalmente, no feedback do usuário.  Utiliza o Google Search
    para auxiliar na identificação e correção de erros.

    Args:
        codigo_qgis (str): O código Python do QGIS gerado pelo Agente Programador.
        log_execucao (str): O log da execução do código, fornecendo informações sobre erros e resultados.
        plano_de_acao (str): O plano de ação detalhado gerado pelo Agente de Interpretação e Planejamento.
        sessao (str, opcional):  Controle de sessão. Se for "reset", zera o contador de versão. Padrão é "Active".
        feedback_usuario (str, opcional): Um texto de feedback do usuário sobre o código ou a execução.
            Pode conter informações sobre erros não detectados no log, sugestões de melhorias, etc.

    Returns:
        str: Um texto contendo instruções claras sobre a natureza do erro, como ele pode ser corrigido e exemplos,
             ou None se a correção não for possível.
    """
    reparador = Agent(
        name="agente_reparador",
        model="gemini-2.0-flash",
        instruction="""
        Você é um especialista em geoprocessamento e depuração de código Python para QGIS. Sua tarefa é analisar o código Python do QGIS fornecido,
        o log de execução, o plano de ação e, opcionalmente, o feedback do usuário, para identificar e corrigir erros no código.

        O código foi gerado pelo Agente Programador a partir de um plano de ação. O log de execução contém
        informações sobre o resultado da execução do código, incluindo erros, avisos e mensagens de progresso.
        O feedback do usuário pode fornecer informações adicionais sobre problemas não detectados no log
        ou sugestões de melhorias.  O plano de ação detalha o que o código deveria fazer.

        Você tem acesso à ferramenta de busca do Google (google_search) para pesquisar erros específicos,
        sintaxe de funções do QGIS, exemplos de código e outras informações relevantes que possam ajudar
        na correção do código.

        Você deve analisar o log de execução, o feedback do usuário (se fornecido) e o plano de ação
        para identificar a causa de quaisquer erros ou problemas no código. Em seguida, você deve usar o
        Google Search para obter mais informações sobre como corrigir esses erros.

        Em vez de gerar um novo código, você deve fornecer instruções claras e concisas sobre a natureza do erro,
        como ele pode ser corrigido e exemplos de código corrigido, se aplicável.  Estruture a resposta de forma
        que seja fácil para um usuário humano entender e aplicar as correções.

        Você deve considerar os seguintes aspectos ao corrigir o código:
        - Erros de sintaxe: Verifique se o código contém erros de sintaxe, como erros de digitação,
          parênteses ou colchetes não correspondentes, etc.
        - Erros de lógica: Verifique se o código contém erros de lógica, como variáveis não inicializadas,
          condições incorretas, loops infinitos, etc.
        - Erros de E/S: Verifique se o código lida corretamente com arquivos de entrada e saída,
          como caminhos de arquivos incorretos, arquivos não encontrados, erros de leitura/escrita, etc.
        - Erros específicos do QGIS: Verifique se o código utiliza as funções e a sintaxe corretas da API do QGIS,
          se lida corretamente com camadas, feições, geometrias, SRC, etc.
        - Exceções: Verifique se o código trata todas as exceções que podem ocorrer durante a execução,
          fornecendo mensagens de erro informativas.
        - Feedback do usuário: Considere o feedback do usuário para corrigir erros não detectados no log,
          implementar sugestões de melhorias ou esclarecer ambiguidades.
        - Plano de Ação: Verifique se o código está aderente ao plano de ação fornecido. Se o código se desviar do plano,
          indique como o código deve ser modificado para seguir o plano corretamente.

        Mantenha uma variável interna chamada 'version_control' que representa a versão do código que você está revisando.
        Incremente essa variável em 1 cada vez que esta função for chamada, a não ser que o parâmetro 'sessao' seja
        explicitamente definido como 'reset'. Se 'sessao' for 'reset', a variável 'version_control' deve ser zerada.
        Em todos os casos, o valor do parâmetro 'sessao' deve ser atualizado para 'Active' ao final da execução da função.

        Inclua uma instrução para o Agente de Q.A. adicionar uma constante no topo do código revisado, contendo
        o valor atual de 'version_control'.
        Exemplo de instrução para o Agente de Q.A.:  "Adicione a seguinte linha no topo do código revisado: `VERSION_CONTROLE = {valor_de_version_control}`"

        Se o código puder ser corrigido, você deve retornar um texto contendo as instruções de correção.
        Se o código não puder ser corrigido, você deve retornar None.

        Exemplo de plano de ação:
        1.  Carregar a camada vetorial 'pontos_relevantes'.
        2.  Iterar sobre todas as feições da camada.
        3.  Para cada feição (ponto), criar um buffer com raio de 500 metros.
        4.  Salvar todas as geometrias de buffer em um novo arquivo shapefile chamado 'buffers_poi.shp'.

        Exemplo de código Python do QGIS:
        ```python
        from qgis.core import QgsVectorLayer, QgsFeature, QgsGeometry, QgsProject
        from qgis.core import QgsField, QgsFields, QgsWkbType
        from qgis.analysis import QgsGeometryProcessing
        import processing
        import os

        def criar_buffer_pontos(input_layer_path, output_path, distancia_buffer):
            \"\"\"
            Cria buffers ao redor de pontos em uma camada vetorial e salva o resultado em um novo arquivo shapefile.

            Args:
                input_layer_path (str): Caminho para a camada vetorial de entrada (pontos).
                output_path (str): Caminho para o arquivo shapefile de saída (buffers).
                distancia_buffer (float): Distância do buffer em metros.
            \"\"\"
            try:
                # 1. Carregar a camada vetorial de entrada
                input_layer = QgsVectorLayer(input_layer_path, "pontos", "ogr")
                if not input_layer.isValid():
                    raise Exception(f"Falha ao carregar a camada vetorial: {input_layer_path}")

                # 2. Verificar se a camada está carregada no projeto
                QgsProject.instance().addMapLayer(input_layer)

                # 3. Criar a camada de saída para os buffers
                crs = input_layer.crs()  # Obter o SRC da camada de entrada
                fields = QgsFields()
                fields.append(QgsField("id", QVariant.Int))  # Adicionar um campo para o ID do ponto original
                output_layer = QgsVectorLayer("Polygon", "buffers", "memory") # Cria camada na memória
                output_layer.setCrs(crs)  # Definir o SRC da camada de saída
                output_layer.dataProvider().addAttributes(fields)
                output_layer.updateFields()

                # 4. Iterar sobre as feições da camada de entrada e criar os buffers
                for feature in input_layer.getFeatures():
                    point_geometry = feature.geometry()
                    buffer_geometry = point_geometry.buffer(distancia_buffer, 5)  # buffer de 5 lados
                    if buffer_geometry is not None: # Verifica se a geometria do buffer é válida
                        buffer_feature = QgsFeature()
                        buffer_feature.setGeometry(buffer_geometry)
                        buffer_feature.setAttributes([feature.id()])  # Atribui o ID do ponto original
                        output_layer.dataProvider().addFeature(buffer_feature)

                # 5. Salvar a camada de buffers em um novo arquivo shapefile
                if QgsVectorFileWriter.writeAsVectorFormat(output_layer, output_path, "utf-8", crs, "ESRI Shapefile") != QgsVectorFileWriter.NoError:
                    raise Exception(f"Falha ao salvar a camada de buffers em: {output_path}")

                # 6. Adicionar a camada de saída ao projeto
                QgsProject.instance().addMapLayer(output_layer)

                print(f"Buffers criados e salvos em: {output_path}")

            except Exception as e:
                print(f"Erro ao criar buffers: {e}")
                return None

        if __name__ == "__main__":
            # Exemplo de uso da função
            input_layer_path = "pontos_relevantes.shp"  # Substitua pelo caminho real da sua camada de pontos
            output_path = "buffers_poi.shp"
            distancia_buffer = 500  # em metros

            # Verificar se o arquivo de entrada existe
            if not os.path.exists(input_layer_path):
                print(f"Erro: Arquivo de entrada não encontrado em: {input_layer_path}")
            else:
                criar_buffer_pontos(input_layer_path, output_path, distancia_buffer)
        ```

        Exemplo de log de execução:
        Log da execução:
        Carregando camada pontos_relevantes.shp...
        Erro: Falha ao carregar a camada vetorial: pontos_relevantes.shp
        Traceback (most recent call last):
          File "<string>", line 10, in <module>
          File "<string>", line 15, in criar_buffer_pontos
        Exception: Falha ao carregar a camada vetorial: pontos_relevantes.shp

        Exemplo de feedback do usuário:
        "O código não funcionou. O arquivo 'pontos_relevantes.shp' não existe no caminho especificado."

        Exemplo de saída (instruções de correção):
        "Erro encontrado: O código não está conseguindo encontrar o arquivo 'pontos_relevantes.shp'.
        Instrução de correção: Verifique se o arquivo existe no caminho especificado. Se o caminho estiver correto,
        verifique se o arquivo não está sendo usado por outro programa.
        Sugestão: Adicione uma verificação no código para garantir que o arquivo existe antes de tentar carregá-lo.
        Exemplo de código corrigido:
        ```python
        if not os.path.exists(input_layer_path):
            raise FileNotFoundError(f'Arquivo não encontrado: {input_layer_path}')
        ```
        Além disso, o código não está tratando o caso em que o arquivo de saída já existe.
        Instrução de correção: Verifique se o arquivo de saída existe antes de tentar salvá-lo. Se existir,
        pergunte ao usuário se ele deseja sobrescrevê-lo ou salvar o resultado em um novo arquivo.
        """,
        description="Agente que corrige código Python do QGIS com base no log de execução, plano de ação e feedback do usuário, usando o Google Search",
        tools=[google_search]  # Adicionando a ferramenta de busca do Google
    )

    # Variável de controle de versão do código
    global version_control
    if 'version_control' not in globals():
        version_control = 1

    if sessao == "reset":
        version_control = 0

    # Garante que a sessão esteja ativa para as próximas chamadas
    sessao = "Active"

    version_control += 1
    # Converte os argumentos para string
    entrada_do_agente_reparador = f"Código QGIS: {codigo_qgis}\nLog de Execução: {log_execucao}\nPlano de Ação: {plano_de_acao}\nFeedback do Usuário: {feedback_usuario}"

    instrucoes_de_correcao = call_agent(reparador, entrada_do_agente_reparador)

    if instrucoes_de_correcao:
        #instrucoes_para_qa = f"Adicione a seguinte linha no topo do código revisado: `VERSION_CONTROL = {version_control}`\n"
        #return instrucoes_para_qa + instrucoes_de_correcao
        instrucoes_para_qa = f"Adicione a seguinte linha no topo do código revisado: `VERSION_CONTROLE = {version_control}`"
        return instrucoes_para_qa + "\n" + instrucoes_de_correcao
    else:
        return None


In [ ]:
# Codigo para testara funcionalidade basica dos agentes. Por hora, rodar os resultados obtidos manualmente no QGIS
comando = input("Digite o que você quer fazer no QGIS hoje:")

if not comando:
    print("Acho que você esqueceu de digitar um comando!")
else:
    print(f"\nMaravilha! Vou fazer a seguinte tarefa para você no QGIS: {comando}. Aguarde...\n")
    PlanoAcao = agente_interpretador(comando)
    #print("\n--- 📝 Resultado do Agente Interpretador ---\n")
    #display(to_markdown(Plano_de_Acao))
    #print("--------------------------------------------------------------")
    Contexto = agente_contextualizador(PlanoAcao,comando)
    #print("\n--- 📝 Resultado do Agente Contextualizador ---\n")
    #display(to_markdown(Contexto))
    #print("--------------------------------------------------------------")
    CodigoPYGIS = agente_programador(PlanoAcao, Contexto)
    #print("\n--- 📝 Resultado do Agente Programador ---\n")
    #display(to_markdown(Codigo_PYGIS))
    #print("--------------------------------------------------------------")
    CodigoREVISADO = agente_avaliador(PlanoAcao, CodigoPYGIS, Contexto, versao_codigo=None)
    #print("\n--- 📝 Resultado do Agente Avaliador ---\n")
    #display(to_markdown(Codigo_REVISADO))
    #print("--------------------------------------------------------------")
    # Nao implementado: agente esta quebrado
    #Sugestoes_Correcao = agente_reparador(Codigo_PYGIS,None, Plano_de_Acao,"Active",None)
    #print("\n--- 📝 Resultado do Agente Reparador ---\n")
    #display(to_markdown(Sugestoes_Correcao))
    #print("--------------------------------------------------------------")
    print("Aqui está o script que implementa a tarefa solicitada. Basta copiar e usar no QGIS. Bom trabalho!\n\n")
    display(to_markdown(CodigoREVISADO))


Digite o que você quer fazer no QGIS hoje:Criar um shape quadrado no centro da tela.

Maravilha! Vou fazer a seguinte tarefa para você no QGIS: Criar um shape quadrado no centro da tela.. Aguarde...

Aqui está o script que implementa a tarefa solicitada. Basta copiar e usar no QGIS. Bom trabalho!




> VERSION_CONTROL = 1
> ```python
> # Importa os módulos necessários do QGIS
> from qgis.core import QgsProject, QgsRectangle, QgsPointXY, QgsFeature, QgsVectorLayer, QgsField, QgsGeometry, QgsWkbTypes, QgsCoordinateReferenceSystem, QgsCoordinateTransform, QgsVectorFileWriter
> from qgis.PyQt.QtCore import QVariant
> from qgis.PyQt.QtWidgets import QInputDialog, QMessageBox
> from qgis.utils import iface
> 
> # Define uma constante de controle de versão
> VERSION_CONTROL = 1
> 
> def criar_quadrado_centralizado():
>     """
>     Cria um quadrado centralizado na extensão do mapa atual do QGIS.
>     """
> 
>     # 1. Obter extensão do mapa atual
>     extent = iface.mapCanvas().extent()
>     xmin = extent.xMinimum()
>     xmax = extent.xMaximum()
>     ymin = extent.yMinimum()
>     ymax = extent.yMaximum()
> 
>     # 2. Calcular o centro da extensão
>     x_center = (xmin + xmax) / 2
>     y_center = (ymin + ymax) / 2
> 
>     # 3. Definir o tamanho do quadrado
>     tamanho_str, ok = QInputDialog.getText(iface.mainWindow(), "Tamanho do Quadrado", "Qual o tamanho desejado do quadrado?")
>     if not ok:
>         return
> 
>     try:
>         tamanho = float(tamanho_str)
>     except ValueError:
>         QMessageBox.critical(iface.mainWindow(), "Erro", "Tamanho inválido. Por favor, insira um número.")
>         return
> 
>     unidade_itens = ("metros", "graus")
>     unidade, ok = QInputDialog.getItem(iface.mainWindow(), "Unidade", "Em que unidade está o tamanho?", unidade_itens, 0, False)
>     if not ok:
>         return
>     
>     # 4. Converter para metros se a unidade for graus
>     if unidade == "graus":
>         # Obter o SRC do projeto
>         project_crs = QgsProject.instance().crs()
> 
>         # Cria um CRS para WGS 84 (EPSG:4326)
>         crs_wgs84 = QgsCoordinateReferenceSystem("EPSG:4326")
> 
>         # Cria uma transformação entre o CRS do projeto e WGS 84
>         transform_context = QgsProject.instance().transformContext()
>         transformation = QgsCoordinateTransform(project_crs, crs_wgs84, transform_context)
> 
>         # Cria um ponto no centro da extensão
>         point = QgsPointXY(x_center, y_center)
> 
>         # Transforma o ponto para WGS 84
>         point_wgs84 = transformation.transform(point)
>         x_wgs84 = point_wgs84.x()
>         y_wgs84 = point_wgs84.y()
> 
>         # Converte a diferença em graus para metros (aproximadamente)
>         tamanho_em_metros_x = tamanho * 111111 * abs(project_crs.mapUnitsPerDegree(point_wgs84)[0]) if project_crs.isGeographic() else tamanho
>         tamanho_em_metros_y = tamanho * 111111 * abs(project_crs.mapUnitsPerDegree(point_wgs84)[1]) if project_crs.isGeographic() else tamanho
>         
>         tamanho = (tamanho_em_metros_x + tamanho_em_metros_y) / 2
> 
>     # 5. Calcular os vértices do quadrado
>     half_size = tamanho / 2
>     ponto1 = QgsPointXY(x_center - half_size, y_center - half_size)
>     ponto2 = QgsPointXY(x_center + half_size, y_center - half_size)
>     ponto3 = QgsPointXY(x_center + half_size, y_center + half_size)
>     ponto4 = QgsPointXY(x_center - half_size, y_center + half_size)
> 
>     # 6. Criar a geometria do quadrado
>     polygon = QgsGeometry.fromPolygonXY([[ponto1, ponto2, ponto3, ponto4, ponto1]])
> 
>     # 7. Criar uma nova camada vetorial
>     layer = QgsVectorLayer("Polygon?crs={}".format(QgsProject.instance().crs().authid()), "quadrado_centralizado", "memory")
>     provider = layer.dataProvider()
> 
>     # Adiciona campos à camada (opcional)
>     provider.addAttributes([QgsField("id", QVariant.Int)])
>     layer.updateFields()
> 
>     # 8. Adicionar o quadrado à camada
>     feature = QgsFeature()
>     feature.setGeometry(polygon)
>     feature.setAttributes([1])
>     provider.addFeatures([feature])
> 
>     # Atualiza a extensão da camada
>     layer.updateExtents()
> 
>     # 9. Salvar a camada
>     nome_arquivo, ok = QInputDialog.getText(iface.mainWindow(), "Salvar Camada", "Digite o nome do arquivo (sem extensão):", text="quadrado_centralizado")
>     if not ok:
>         return
> 
>     caminho_arquivo = "/tmp/" + nome_arquivo + ".shp"  # Ou outro diretório desejado
>     
>     writer_options = QgsVectorFileWriter.SaveVectorOptions()
>     writer_options.driverName = "ESRI Shapefile"
> 
>     error = QgsVectorFileWriter.writeAsVectorFormat(layer, caminho_arquivo, "utf-8", layer.crs(), options=writer_options)
>     
>     if error[0] == QgsVectorFileWriter.NoError:
>         QgsProject.instance().addMapLayer(layer)
>         iface.messageBar().pushMessage("Sucesso", "Quadrado centralizado criado e salvo em: {}".format(caminho_arquivo), level=Qgis.MessageLevel.Info)
>     else:
>         QMessageBox.critical(iface.mainWindow(), "Erro ao salvar camada", "Erro ao salvar: {}".format(error[1]))
> 
> # Executa a função principal
> criar_quadrado_centralizado()
> ```
